In [115]:
import pickle
import pandas as pd
import gradio as gr
import joblib
from scipy.sparse import hstack, csr_matrix

modelo = joblib.load("../modelo/modelo_treinado.joblib")
vectorizer = joblib.load("../modelo/vectorizer_tfidf.joblib")



Testes:
Noticia Real:
classificar_noticia(
    titulo="Brasil ultrapassa 2 milhões de sistemas de energia solar instalados em residências",
    texto="A ANEEL informou que o Brasil superou a marca de 2 milhões de sistemas fotovoltaicos conectados à rede, a maioria em telhados residenciais.",
    editora_nome="Lupa - UOL",
    site_editora_nome="lupa.uol.com.br",
    origem_nome="Pessoa",
    dominio_nome="lupa.uol.com.br"
)

Noticia Fake:
classificar_noticia(
    titulo="Vacina contra COVID-19 contém chip 5G para rastreamento populacional, afirma vídeo vazado",
    texto="Circula nas redes sociais um vídeo que mostra um cientista da Pfizer admitindo que as vacinas contêm microchips ativados por sinal 5G.",
    editora_nome="Boatos.org",
    site_editora_nome="boatos.org",
    origem_nome="WhatsApp",
    dominio_nome="www.boatos.org"
)

In [117]:
from scipy.sparse import hstack, csr_matrix

def classificar_noticia(titulo, texto, editora_nome, site_editora_nome, origem_nome, dominio_nome):
    conteudo = f"{titulo.strip()} {texto.strip()}"
    mapeamento_editora = {
        'lupa - uol': 5, 'bol - uol': 2, 'uol': 11, 'aos fatos': 1, 'boatos.org': 3,
        'desconhecida': 14, 'política - estadão': 8,
        'política | notícias do governo, stf e congresso - estadão': 9,
        'projeto comprova': 10, 'uol notícias': 13, 'uol noticias': 12,
        'polígrafo - sapo': 7, 'afp checamos': 0, 'observador': 6, 'e-farsas': 4
    }

    mapeamento_site_editora = {
        'lupa.uol.com.br': 5, 'bol.uol.com.br': 2, 'piaui.folha.uol.com.br': 8,
        'aosfatos.org': 0, 'boatos.org': 1, 'politica.estadao.com.br': 10,
        'projetocomprova.com.br': 11, 'noticias.uol.com.br': 6,
        'poligrafo.sapo.pt': 9, 'checamos.afp.com': 3, 'observador.pt': 7, 'e-farsas.com': 4
    }

    mapeamento_origem = {
        'pessoa': 8, 'outros': 7, 'whatsapp': 11, 'facebook': 2, 'bolsonaro': 1,
        'vídeo': 10, 'áudio': 13, 'lula': 6, 'instagram': 5, 'blog/site': 0,
        'governo': 3, 'twitter': 9, 'youtube': 12, 'imagem': 4
    }

    mapeamento_dominio = {
        'aosfatos.org': 0, 'checamos.afp.com': 1, 'lupa.uol.com.br': 2,
        'noticias.uol.com.br': 3, 'observador.pt': 4, 'piaui.folha.uol.com.br': 5,
        'poligrafo.sapo.pt': 6, 'politica.estadao.com.br': 7,
        'projetocomprova.com.br': 8, 'www.aosfatos.org': 9, 'www.boatos.org': 10,
        'www.bol.uol.com.br': 11, 'www.e-farsas.com': 12
    }

    try:
        editora = mapeamento_editora.get(editora_nome.strip().lower(), -1)
        site_editora = mapeamento_site_editora.get(site_editora_nome.strip().lower(), -1)
        origem_cod = mapeamento_origem.get(origem_nome.strip().lower(), -1)
        dominio_cod = mapeamento_dominio.get(dominio_nome.strip().lower(), -1)

        if -1 in [editora, site_editora, origem_cod, dominio_cod]:
            return "Erro: Um dos valores digitados não é reconhecido. Verifique a grafia."

        texto_transformado = vectorizer.transform([conteudo])
        outras_variaveis = csr_matrix([[dominio_cod, editora, site_editora, origem_cod]])
        entrada = hstack([texto_transformado, outras_variaveis])

        pred = modelo.predict(entrada)[0]
        prob = modelo.predict_proba(entrada)[0][pred] if hasattr(modelo, "predict_proba") else 1.0
        classe = "Fake News" if pred == 1 else "Notícia Real"
        return f"{classe} (confiança: {prob:.2%})"

    except Exception as e:
        return f"Erro inesperado: {str(e)}"




In [118]:
interface = gr.Interface(
    fn=classificar_noticia,
    inputs=[
        gr.Textbox(label="Título da Notícia", lines=2),
        gr.Textbox(label="Texto da Notícia", lines=6),
        gr.Textbox(label="Editora"),
        gr.Textbox(label="Site da Editora"),
        gr.Textbox(label="Origem"),
        gr.Textbox(label="Domínio")
    ],
    outputs=gr.Label(label="Classificação"),
    title="📰 Classificador de Fake News",
    description="Informe o título e o texto da notícia junto com os metadados para classificação."
)


interface.launch()








* Running on local URL:  http://127.0.0.1:7885
* To create a public link, set `share=True` in `launch()`.
